In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [3]:
def loadDatasetRaw(name, encoding="utf-8", print=False): #"ISO-8859-1"
  df = pd.read_csv(f"../raw_data/{name}.csv", encoding=encoding)
  if(print):
    print(df.info())
    print(df.head())
  return df

def loadDataset(name, encoding="utf-8", print=False): #"ISO-8859-1"
  df = pd.read_csv(f"{name}.csv", encoding=encoding)
  if(print):
    print(df.info())
    print(df.head())
  return df

In [5]:
airportList = loadDatasetRaw("airports_list")
passengers = loadDatasetRaw("T3_AIR_CARRIER_SUMMARY_AIRPORT_ACTIVITY_2019")
aircraft = loadDatasetRaw("B43_AIRCRAFT_INVENTORY", encoding='latin1')
aircraft.drop_duplicates(subset='TAIL_NUM', inplace=True)
coords = loadDatasetRaw("AIRPORT_COORDINATES")
coords.drop_duplicates(subset='ORIGIN_AIRPORT_ID', inplace=True)
names = loadDatasetRaw("CARRIER_DECODE")
names.drop_duplicates(inplace=True)
names.drop_duplicates(subset=['OP_UNIQUE_CARRIER'], inplace=True)
employees = loadDatasetRaw("P10_EMPLOYEES")
employees = employees[['OP_UNIQUE_CARRIER', 'PASS_GEN_SVC_ADMIN', 'PASSENGER_HANDLING']]
employees = employees.groupby('OP_UNIQUE_CARRIER').sum().reset_index()

In [ ]:
def month_cleanup(monthly_data, aircraft, coords, names, weather, passengers, employees):
            
    # CLEANING
    # drop rows with no departure time, tail number, or were cancelled
    print("Dropping NaNs from Dep Time, Tail Num. Dropping Cancellations.")
    monthly_data.drop(monthly_data.loc[monthly_data['DEP_TIME'].isna()].index, axis=0, inplace=True)
    monthly_data.drop(monthly_data.loc[monthly_data['TAIL_NUM'].isna()].index, axis=0, inplace=True)
    monthly_data.drop(monthly_data.loc[monthly_data['CANCELLED']==1].index, axis=0, inplace=True)
 
    # FEATURE ENGINEERING - SEGMENT NUMBER
    # List flight segment number for daily flight segments by tracking tail number
    print("Adding Flight Number Sequence - SEGMENT_NUMBER")
    monthly_data["SEGMENT_NUMBER"] = monthly_data.groupby(["TAIL_NUM", 'DAY_OF_MONTH'])["DEP_TIME"].rank("dense", ascending=True)
    
    # FEATURE ENGINEERING - CONCURRENT FLIGHTS
    # Listing the number of concurrent flights at the airport in the time block 
    print("Adding Concurrent Flights - CONCURRENT_FLIGHTS")
    monthly_data['CONCURRENT_FLIGHTS'] = monthly_data.groupby(['ORIGIN_AIRPORT_ID','DAY_OF_MONTH', 'DEP_TIME_BLK'])['OP_UNIQUE_CARRIER'].transform("count")
 
    # MERGING to get NUMBER_OF_SEATS
    print("Applying seat counts to flights - NUMBER_OF_SEATS")   
    # Merge aircraft info with main frame on tail number - get NUMBER_OF_SEATS 
    monthly_data = pd.merge(monthly_data, aircraft, how="left", on='TAIL_NUM')
    # Fill missing aircraft info with means
    monthly_data['NUMBER_OF_SEATS'].fillna((monthly_data['NUMBER_OF_SEATS'].mean()), inplace=True)
    # simplify data type of number of seats to reduce memory usage
    monthly_data['NUMBER_OF_SEATS'] = monthly_data['NUMBER_OF_SEATS'].astype('int16')

    # MERGING
    # Merge to get proper carrier name
    print("Applying Carrier Names - CARRIER_NAME")  
    monthly_data = pd.merge(monthly_data, names, how='left', on=['OP_UNIQUE_CARRIER'])
    
    # FEATURE ENGINEERING - AIRPORT_FLIGHTS_MONTH, AIRLINE_FLIGHTS_MONTH, AIRLINE_AIRPORT_FLIGHTS_MONTH
    # Add monthly flight statistics for carrier and airport
    print("Adding flight statistics for carrier and airport - AIRPORT_FLIGHTS_MONTH, AIRLINE_FLIGHTS_MONTH, AIRLINE_AIRPORT_FLIGHTS_MONTH")
    monthly_data['AIRPORT_FLIGHTS_MONTH'] = monthly_data.groupby(['ORIGIN_AIRPORT_ID'])['ORIGIN_CITY_NAME'].transform('count')
    monthly_data['AIRLINE_FLIGHTS_MONTH'] = monthly_data.groupby(['OP_UNIQUE_CARRIER'])['ORIGIN_CITY_NAME'].transform('count')
    monthly_data['AIRLINE_AIRPORT_FLIGHTS_MONTH'] = monthly_data.groupby(['OP_UNIQUE_CARRIER', 'ORIGIN_AIRPORT_ID'])['ORIGIN_CITY_NAME'].transform('count')
    
    # FEATURE ENGINEERING - AVG_MONTHLY_PASS_AIRPORT, AVG_MONTHLY_PASS_AIRLINE
    #Add monthly passenger statistics for carrier and airport
    print("Adding passenger statistics for carrier and airport - AVG_MONTHLY_PASS_AIRPORT, AVG_MONTHLY_PASS_AIRLINE")
    monthly_airport_passengers = pd.DataFrame(passengers.groupby(['ORIGIN_AIRPORT_ID'])['REV_PAX_ENP_110'].sum())
    monthly_data = pd.merge(monthly_data, monthly_airport_passengers, how='left', on=['ORIGIN_AIRPORT_ID'])
    monthly_data['AVG_MONTHLY_PASS_AIRPORT'] = (monthly_data['REV_PAX_ENP_110']/12).astype('int64')
    monthly_airline_passengers = pd.DataFrame(passengers.groupby(['OP_UNIQUE_CARRIER'])['REV_PAX_ENP_110'].sum())
    monthly_data = pd.merge(monthly_data, monthly_airline_passengers, how='left', on=['OP_UNIQUE_CARRIER'])
    monthly_data['AVG_MONTHLY_PASS_AIRLINE'] = (monthly_data['REV_PAX_ENP_110_y']/12).astype('int64')
    
    # MERGING
    # Add employee stats then FEATURE ENGINEER FLT_ATTENDANTS_PER_PASS, GROUND_SERV_PER_PASS
    print("Adding employee statistics for carrier - FLT_ATTENDANTS_PER_PASS, GROUND_SERV_PER_PASS")
    monthly_data = pd.merge(monthly_data, employees, how='left', on=['OP_UNIQUE_CARRIER'])
    monthly_data['FLT_ATTENDANTS_PER_PASS'] = monthly_data['PASSENGER_HANDLING']/monthly_data['REV_PAX_ENP_110_y']
    monthly_data['GROUND_SERV_PER_PASS'] = monthly_data['PASS_GEN_SVC_ADMIN']/monthly_data['REV_PAX_ENP_110_y']
    
    # FEATURE ENGINEERING - PLANE AGE
    # Calculate age of plane
    print("Calculate Fleet Age - PLANE_AGE")
    monthly_data['MANUFACTURE_YEAR'].fillna((monthly_data['MANUFACTURE_YEAR'].mean()), inplace=True)
    monthly_data['PLANE_AGE'] = 2019 - monthly_data['MANUFACTURE_YEAR']

    # MERGING
    # Merge to get airport coordinates
    print("Adding airport coordinates - LATITUDE, LONGITUDE, DEPARTING_AIRPORT")
    monthly_data = pd.merge(monthly_data, coords, how='left', on=['ORIGIN_AIRPORT_ID'])
    monthly_data['LATITUDE'] = round(monthly_data['LATITUDE'], 3)
    monthly_data['LONGITUDE'] = round(monthly_data['LONGITUDE'], 3)

    # FEATURE ENGINEERING - PREVIOUS AIRPORT
    # Get previous airport for tail number
    print("Adding airports - PREVIOUS_AIRPORT")
    segment_temp = monthly_data[['DAY_OF_MONTH', 'TAIL_NUM', 'DISPLAY_AIRPORT_NAME', 'SEGMENT_NUMBER']]
    monthly_data = pd.merge_asof(monthly_data.sort_values('SEGMENT_NUMBER'), segment_temp.sort_values('SEGMENT_NUMBER'), on='SEGMENT_NUMBER', by=['DAY_OF_MONTH', 'TAIL_NUM'], allow_exact_matches=False)
    monthly_data['DISPLAY_AIRPORT_NAME_y'].fillna('NONE', inplace=True)
    monthly_data.rename(columns={"DISPLAY_AIRPORT_NAME_y": "PREVIOUS_AIRPORT", "DISPLAY_AIRPORT_NAME_x": "DEPARTING_AIRPORT"}, inplace=True)  
    
    # CLEANING  
    # Drop airports below the 10th percentile
    print("Dropping bottom 10% of airports")
    monthly_data.drop(monthly_data.loc[monthly_data['AIRPORT_FLIGHTS_MONTH'] < 1100].index, axis=0, inplace=True)
    
    # MERGING
    # Merge weather data
    print("Adding daily weather data - PRCP, SNOW, SNWD, SMAX, TMIN, AWND")
    monthly_data = pd.merge(monthly_data, weather, how='inner', on=['ORIGIN_AIRPORT_ID', 'MONTH', 'DAY_OF_MONTH'])

    
    # CLEANING
    # drop columns that we won't use
    print("Clean up unneeded columns")
    monthly_data.drop(columns = ['ORIGIN',  'DEST',  
                   'CRS_DEP_TIME', 'CRS_ARR_TIME', 'CANCELLED', 'CANCELLATION_CODE', 'CRS_ELAPSED_TIME', 'DISTANCE',
                   'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY',
                  'Unnamed: 32',  'ACTUAL_ELAPSED_TIME', 'DEST_AIRPORT_ID', 'DEST_CITY_NAME',  'OP_CARRIER_FL_NUM',  'OP_UNIQUE_CARRIER',
                       'AIRLINE_ID', 'DATE', 'DAY_OF_MONTH', 'TAIL_NUM', 'ORIGIN_AIRPORT_ID', 'ORIGIN_CITY_NAME',  
                       'PASSENGER_HANDLING', 'REV_PAX_ENP_110_x', 'REV_PAX_ENP_110_y', 
                                 'PASS_GEN_SVC_ADMIN', 'MANUFACTURE_YEAR',
                                 ],
                    axis=1, inplace=True)
    
    # CLEANING
    # specify data types of various fields to reduce memory usage
    print("Cleaning up data types")
    monthly_data['MONTH'] = monthly_data['MONTH'].astype('object')
    monthly_data['DAY_OF_WEEK'] = monthly_data['DAY_OF_WEEK'].astype('object')
    monthly_data['DEP_DEL15'] = monthly_data['DEP_DEL15'].astype('int8')
    monthly_data['DISTANCE_GROUP'] = monthly_data['DISTANCE_GROUP'].astype('int8')
    monthly_data['SEGMENT_NUMBER'] = monthly_data['SEGMENT_NUMBER'].astype('int8')
    monthly_data['AIRPORT_FLIGHTS_MONTH'] = monthly_data['AIRPORT_FLIGHTS_MONTH'].astype('int64')
    monthly_data['AIRLINE_FLIGHTS_MONTH'] = monthly_data['AIRLINE_FLIGHTS_MONTH'].astype('int64')
    monthly_data['AIRLINE_AIRPORT_FLIGHTS_MONTH'] = monthly_data['AIRLINE_AIRPORT_FLIGHTS_MONTH'].astype('int64')
    monthly_data['PLANE_AGE'] = monthly_data['PLANE_AGE'].astype('int32')
    
    # reset index
    monthly_data.reset_index(inplace=True, drop=True)
        
    print("FINISHED")
    
    # return cleaned file
    return monthly_data

In [12]:
weather = loadDatasetRaw("airport_weather_2019")
weather_merge = pd.merge(airportList, weather, how='left', on='NAME')
weather = weather_merge[['DATE', 'PRCP', 'SNOW', 'SNWD', 'TMAX', 'AWND', 'ORIGIN_AIRPORT_ID']]
weather.drop(weather.loc[weather['ORIGIN_AIRPORT_ID'].isna()].index, axis=0, inplace=True)
print(weather.isna().sum())
print(weather['DATE'].count())
weather['TMAX'].fillna(round(weather.groupby('ORIGIN_AIRPORT_ID')['TMAX'].transform('mean'), 1), inplace=True)
weather['AWND'].fillna(round(weather.groupby('ORIGIN_AIRPORT_ID')['AWND'].transform('mean'), 1), inplace=True)
weather.fillna(0, inplace=True)
weather['DATE'] = pd.to_datetime(weather['DATE'], format="mixed")
weather['MONTH'] = pd.DatetimeIndex(weather['DATE']).month
weather['DAY_OF_MONTH'] = pd.DatetimeIndex(weather['DATE']).day
weather.head()

DATE                     1
PRCP                    22
SNOW                 11060
SNWD                 11565
TMAX                     6
AWND                     9
ORIGIN_AIRPORT_ID        0
dtype: int64
35024


,DATE,PRCP,SNOW,SNWD,TMAX,AWND,ORIGIN_AIRPORT_ID,MONTH,DAY_OF_MONTH
0,2019-01-01,0.00,0.0,0.0,45.0,4.70,12992,1,1
1,2019-01-02,0.39,0.0,0.0,39.0,2.01,12992,1,2
2,2019-01-03,0.44,0.0,0.0,41.0,6.26,12992,1,3
3,2019-01-04,0.13,0.0,0.0,47.0,2.01,12992,1,4
4,2019-01-05,0.00,0.0,0.0,62.0,1.79,12992,1,5


In [ ]:
df = loadDatasetRaw("ONTIME_REPORTING_01")
month01 = month_cleanup(df, aircraft, coords, names, weather, passengers, employees)
df = loadDatasetRaw("ONTIME_REPORTING_02")
month02 = month_cleanup(df, aircraft, coords, names, weather, passengers, employees)
df = loadDatasetRaw("ONTIME_REPORTING_03")
month03 = month_cleanup(df, aircraft, coords, names, weather, passengers, employees)
df = loadDatasetRaw("ONTIME_REPORTING_04")
month04 = month_cleanup(df, aircraft, coords, names, weather, passengers, employees)
df = loadDatasetRaw("ONTIME_REPORTING_05")
month05 = month_cleanup(df, aircraft, coords, names, weather, passengers, employees)
df = loadDatasetRaw("ONTIME_REPORTING_06")
month06 = month_cleanup(df, aircraft, coords, names, weather, passengers, employees)
df = loadDatasetRaw("ONTIME_REPORTING_07")
month07 = month_cleanup(df, aircraft, coords, names, weather, passengers, employees)
df = loadDatasetRaw("ONTIME_REPORTING_08")
month08 = month_cleanup(df, aircraft, coords, names, weather, passengers, employees)
df = loadDatasetRaw("ONTIME_REPORTING_09")
month09 = month_cleanup(df, aircraft, coords, names, weather, passengers, employees)
df = loadDatasetRaw("ONTIME_REPORTING_10")
month10 = month_cleanup(df, aircraft, coords, names, weather, passengers, employees)
df = loadDatasetRaw("ONTIME_REPORTING_11")
month11 = month_cleanup(df, aircraft, coords, names, weather, passengers, employees)
df = loadDatasetRaw("ONTIME_REPORTING_12")
month12 = month_cleanup(df, aircraft, coords, names, weather, passengers, employees)

# COMBINE MASTER FILE

all_data = pd.concat([month01, month02, month03, month04, month05, month06, month07, month08, month09, month10, month11, month12]).reset_index(drop=True)
all_data.to_csv('data/raw_dataset/datasetWithMoreData.csv', index=False)

In [4]:
df = pd.read_csv("data/raw_dataset/train_val.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6489062 entries, 0 to 6489061
Data columns (total 26 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   MONTH                          int64  
 1   DAY_OF_WEEK                    int64  
 2   DEP_DEL15                      int64  
 3   DEP_TIME_BLK                   object 
 4   DISTANCE_GROUP                 int64  
 5   SEGMENT_NUMBER                 int64  
 6   CONCURRENT_FLIGHTS             int64  
 7   NUMBER_OF_SEATS                int64  
 8   CARRIER_NAME                   object 
 9   AIRPORT_FLIGHTS_MONTH          int64  
 10  AIRLINE_FLIGHTS_MONTH          int64  
 11  AIRLINE_AIRPORT_FLIGHTS_MONTH  int64  
 12  AVG_MONTHLY_PASS_AIRPORT       int64  
 13  AVG_MONTHLY_PASS_AIRLINE       int64  
 14  FLT_ATTENDANTS_PER_PASS        float64
 15  GROUND_SERV_PER_PASS           float64
 16  PLANE_AGE                      int64  
 17  DEPARTING_AIRPORT              object 
 18  LA